# Part 4: Using GPU acceleration with PyTorch

In [1]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try:
    import torchbearer
except:
    !pip install torchbearer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.1/138.1 kB 3.7 MB/s eta 0:00:00


## Manual use of `.cuda()`

Now the magic of PyTorch comes in. So far, we've only been using the CPU to do computation. When we want to scale to a bigger problem, that won't be feasible for very long.
|
PyTorch makes it really easy to use the GPU for accelerating computation. Consider the following code that computes the element-wise product of two large matrices:

In [4]:
import torch

t1 = torch.randn(1000, 1000)
t2 = torch.randn(1000, 1000)
t3 = t1*t2
print(t3)

tensor([[-0.3156,  0.0280,  0.0214,  ..., -0.5833, -0.5859,  1.0810],
        [ 0.5394,  0.0042, -0.1461,  ...,  0.0832,  0.2621,  0.2351],
        [-0.5285,  1.5096, -0.2568,  ...,  0.2057,  0.4414, -0.2022],
        ...,
        [ 0.0344, -0.1287,  0.0086,  ..., -0.5125, -1.7514,  2.2217],
        [ 0.5815, -1.6087,  0.5825,  ...,  1.0619, -1.9444,  0.5200],
        [-0.5233,  2.3536,  0.2135,  ...,  0.2228,  1.7458, -0.0094]])


By sending all the tensors that we are using to the GPU, all the operations on them will also run on the GPU without having to change anything else. If you're running a non-cuda enabled version of PyTorch the following will throw an error; if you have cuda available the following will create the input matrices, copy them to the GPU and perform the multiplication on the GPU itself:

In [5]:
t1 = torch.randn(1000, 1000).cuda()
t2 = torch.randn(1000, 1000).cuda()
t3 = t1*t2
print(t3)

tensor([[-0.1878,  0.1184, -3.1167,  ...,  0.3020, -2.0150, -0.4981],
        [ 0.1620,  0.3612,  0.9119,  ..., -0.2658, -1.7895,  2.9992],
        [-0.8685,  0.9108,  1.3363,  ..., -0.2263, -1.4499,  0.6814],
        ...,
        [-1.0709,  0.1945,  0.1628,  ..., -2.7508,  0.4809,  0.2114],
        [-0.1397, -0.7006, -0.0232,  ...,  0.4216,  0.0135,  1.6060],
        [-0.7458,  0.3254,  0.2455,  ...,  1.5217, -0.3073,  0.4970]],
       device='cuda:0')


If you're running this workbook in colab, now enable GPU acceleration (`Runtime->Runtime Type` and add a `GPU` in the hardware accelerator pull-down). You'll then need to re-run all cells to this point.

If you were able to run the above with hardware acceleration, the print-out of the result tensor would show that it was an instance of `cuda.FloatTensor` type on the the `(GPU 0)` GPU device. If your wanted to copy the tensor back to the CPU, you would use the `.cpu()` method.

## Writing platform agnostic code

Most of the time you'd like to write code that is device agnostic; that is it will run on a GPU if one is available, and otherwise it would fall back to the CPU. The recommended way to do this is as follows:

In [6]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t1 = torch.randn(1000, 1000).to(device)
t2 = torch.randn(1000, 1000).to(device)
t3 = t1*t2
print(t3)

tensor([[-1.6681, -0.3570, -0.9255,  ...,  0.0506, -0.4491, -1.7188],
        [-1.4889,  0.8528,  0.8919,  ...,  1.8131,  0.0937,  0.2719],
        [ 1.3700, -0.3513, -0.0142,  ..., -0.3089, -0.1183,  0.1354],
        ...,
        [ 0.2589, -0.3988, -3.3979,  ...,  0.0692, -0.2008,  2.0483],
        [-1.5438, -0.0753, -2.7590,  ..., -0.3005, -1.0377, -0.5898],
        [ 0.0538,  0.9572,  0.4833,  ...,  0.0157, -0.4148,  0.1388]],
       device='cuda:0')


## Accelerating neural net training

If you wanted to accelerate the training of a neural net using raw PyTorch, you would have to copy both the model and the training data to the GPU. Unless you were using a really small dataset like MNIST, you would typically _stream_ the batches of training data to the GPU as you used them in the training loop:

```python
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = BaselineModel(784, 784, 10).to(device)

loss_function = ...
optimiser = ...

for epoch in range(10):
    for data in trainloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimiser.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()
```

Using Torchbearer, this becomes much simpler - you just tell the `Trial` to run on the GPU and that's it!:

```python
model = BetterCNN()

loss_function = ...
optimiser = ...

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader)
trial.run(epochs=10)
```


## Multiple GPUs

Using multiple GPUs is beyond the scope of the lab, but if you have multiple cuda devices, they can be referred to by index: `cuda:0`, `cuda:1`, `cuda:2`, etc. You have to be careful not to mix operations on different devices, and would need how to carefully orchestrate moving of data between the devices (which can really slow down your code to the point at which using the CPU would actually be faster).